BiLSTM+CRF模型训练，使用100维随机初始化词嵌入

In [1]:
# 显卡查看
! nvidia-smi

Thu May 21 01:15:26 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
# 依赖安装
! pip install fastNLP

加载数据集

In [3]:
import sys

from fastNLP.core import Const
from fastNLP.io import PeopleDailyNERLoader
from fastNLP.io import PeopleDailyPipe

sys.path.insert(0, '/content/drive/My Drive/my_framework/qyt_clue/')  # 定义搜索路径的优先顺序，序号从0开始，表示最大优先级

import myClue  # noqa
print('myClue module path :{}'.format(myClue.__file__))  # 输出测试模块文件位置
from myClue.core import logger  # noqa
from myClue.core.utils import print_data_bundle  # noqa
from myClue.tools.serialize import save_serialize_obj  # noqa

myClue module path :/content/drive/My Drive/my_framework/qyt_clue/myClue/__init__.py


In [0]:
# 原始语料
# train_file_config = {
#     'train': '/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/train.conll',
#     'dev': '/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/dev.conll',
#     'test': '/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/test.conll',
# }

In [0]:
# 使用最佳模型预测后的语料
train_file_config = {
    'train': '/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/train_bert_predict.conll',
    'dev': '/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/dev_bert_predict.conll',
    'test': '/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/test.conll',
}

In [6]:
# train_data_bundle_pkl_file = './data/weibo_NER/train_data_bundle.pkl'
logger.info('数据加载')
data_loader = PeopleDailyNERLoader()
data_bundle = data_loader.load(train_file_config)
print_data_bundle(data_bundle)
logger.info('数据预处理')
data_pipe = PeopleDailyPipe()
data_bundle = data_pipe.process(data_bundle)
data_bundle.rename_field(field_name=Const.CHAR_INPUT, new_field_name=Const.INPUT, ignore_miss_dataset=True, rename_vocab=True)
print_data_bundle(data_bundle)
# save_serialize_obj(data_bundle, train_data_bundle_pkl_file)
# logger.info('数据预处理后进行序列化：{}'.format(train_data_bundle_pkl_file))

2020-05-21 01:15:33 I [<ipython-input-6-a24be1271d63>:2] 数据加载
2020-05-21 01:15:33 I [utils.py:16] dataset name : train
2020-05-21 01:15:33 I [utils.py:17] dataset len : 1350
2020-05-21 01:15:33 I [utils.py:18] dataset example : 
2020-05-21 01:15:33 I [utils.py:19] 
+------------------------------------------+------------------------------------------+
| raw_chars                                | target                                   |
+------------------------------------------+------------------------------------------+
| ['科', '技', '全', '方', '位', '资', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
| ['对', '，', '输', '给', '一', '个', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'B-PER... |
| ['今', '天', '下', '午', '起', '来', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
| ['今', '年', '拜', '年', '不', '短', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
| ['浑', '身', '酸', '疼', '，', '两', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
+------------------------------------------+----------------

+-------------+-----------+--------+
| field_names | raw_chars | target |
+-------------+-----------+--------+
|   is_input  |   False   | False  |
|  is_target  |   False   | False  |
| ignore_type |           |        |
|  pad_value  |           |        |
+-------------+-----------+--------+
+-------------+-----------+--------+
| field_names | raw_chars | target |
+-------------+-----------+--------+
|   is_input  |   False   | False  |
|  is_target  |   False   | False  |
| ignore_type |           |        |
|  pad_value  |           |        |
+-------------+-----------+--------+
+-------------+-----------+--------+
| field_names | raw_chars | target |
+-------------+-----------+--------+
|   is_input  |   False   | False  |
|  is_target  |   False   | False  |
| ignore_type |           |        |
|  pad_value  |           |        |
+-------------+-----------+--------+


2020-05-21 01:15:34 I [utils.py:16] dataset name : train
2020-05-21 01:15:34 I [utils.py:17] dataset len : 1350
2020-05-21 01:15:34 I [utils.py:18] dataset example : 
2020-05-21 01:15:34 I [utils.py:19] 
+------------------------+------------------------+---------+------------------------+
| raw_chars              | target                 | seq_len | words                  |
+------------------------+------------------------+---------+------------------------+
| ['科', '技', '全', ... | [0, 0, 0, 0, 0, 0, ... | 26      | [792, 1015, 156, 19... |
| ['对', '，', '输', ... | [0, 0, 0, 0, 0, 0, ... | 15      | [123, 2, 1205, 115,... |
| ['今', '天', '下', ... | [0, 0, 0, 0, 0, 0, ... | 79      | [164, 33, 65, 739, ... |
| ['今', '年', '拜', ... | [0, 0, 0, 0, 0, 0, ... | 18      | [164, 92, 880, 92, ... |
| ['浑', '身', '酸', ... | [0, 0, 0, 0, 0, 0, ... | 18      | [2072, 221, 1056, 9... |
+------------------------+------------------------+---------+------------------------+
2020-05-21 01:15:34 I [utils

+-------------+-----------+--------+---------+-------+
| field_names | raw_chars | target | seq_len | words |
+-------------+-----------+--------+---------+-------+
|   is_input  |   False   |  True  |   True  |  True |
|  is_target  |   False   |  True  |   True  | False |
| ignore_type |           | False  |  False  | False |
|  pad_value  |           |   0    |    0    |   0   |
+-------------+-----------+--------+---------+-------+
+-------------+-----------+--------+---------+-------+
| field_names | raw_chars | target | seq_len | words |
+-------------+-----------+--------+---------+-------+
|   is_input  |   False   |  True  |   True  |  True |
|  is_target  |   False   |  True  |   True  | False |
| ignore_type |           | False  |  False  | False |
|  pad_value  |           |   0    |    0    |   0   |
+-------------+-----------+--------+---------+-------+
+-------------+-----------+--------+---------+-------+
| field_names | raw_chars | target | seq_len | words |
+---------

模型训练

In [0]:
import os
import sys
import time

import torch

from fastNLP import Const
from fastNLP.embeddings import StaticEmbedding
from fastNLP.models import BiLSTMCRF
from fastNLP import SpanFPreRecMetric
from fastNLP import Trainer
from fastNLP import LossInForward
from torch.optim import Adam
from fastNLP import Tester

from myClue.core import logger  # noqa
from myClue.core.callback import EarlyStopCallback  # noqa
from myClue.tools.serialize import load_serialize_obj  # noqa
from myClue.tools.serialize import save_serialize_obj  # noqa
from myClue.tools.file import init_file_path  # noqa

In [8]:
# train_data_bundle_pkl_file = '/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/train_data_bundle.pkl'
model_path = '/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/model_bilstm_crf_random_embed'
init_file_path(model_path)
logger.add_file_handler(os.path.join(model_path, 'log_{}.txt'.format(time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime()))))  # 日志写入文件
char_vocab_pkl_file = os.path.join(model_path, 'vocab_char.pkl')
target_vocab_pkl_file = os.path.join(model_path, 'target_char.pkl')
# logger.warn('加载数据集')
# data_bundle = load_serialize_obj(train_data_bundle_pkl_file)
logger.warn('获取词典')
char_vocab = data_bundle.get_vocab('words')
logger.info('char_vocab:{}'.format(char_vocab))
target_vocab = data_bundle.get_vocab('target')
logger.info('target_vocab:{}'.format(target_vocab))
save_serialize_obj(char_vocab, char_vocab_pkl_file)
save_serialize_obj(target_vocab, target_vocab_pkl_file)
logger.info('词典序列化:{}'.format(char_vocab_pkl_file))
logger.warn('选择预训练词向量')
word2vec_embed = StaticEmbedding(char_vocab, model_dir_or_name=None, embedding_dim=100)
logger.warn('神经网络模型')
model = BiLSTMCRF(embed=word2vec_embed, num_classes=len(target_vocab), num_layers=1, hidden_size=200, dropout=0.5,
                  target_vocab=target_vocab)
logger.info(model)
logger.warn('训练超参数设定')
loss = LossInForward()
optimizer = Adam([param for param in model.parameters() if param.requires_grad])
# metric = AccuracyMetric()
metric = SpanFPreRecMetric(tag_vocab=data_bundle.get_vocab(Const.TARGET), only_gross=False)  # 若only_gross=False, 即还会返回各个label的metric统计值
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # 如果有gpu的话在gpu上运行，训练速度会更快
logger.info('device:{}'.format(device))

2020-05-21 01:15:34 W [<ipython-input-8-0bb789491088>:9] 获取词典
2020-05-21 01:15:34 I [<ipython-input-8-0bb789491088>:11] char_vocab:Vocabulary(['科', '技', '全', '方', '位']...)
2020-05-21 01:15:34 I [<ipython-input-8-0bb789491088>:13] target_vocab:Vocabulary(['O', 'B-PER.NOM', 'I-PER.NOM', 'B-LOC.NAM', 'I-LOC.NAM']...)
2020-05-21 01:15:34 I [<ipython-input-8-0bb789491088>:16] 词典序列化:/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/model_bilstm_crf_random_embed/vocab_char.pkl
2020-05-21 01:15:34 W [<ipython-input-8-0bb789491088>:17] 选择预训练词向量
2020-05-21 01:15:34 W [<ipython-input-8-0bb789491088>:19] 神经网络模型
2020-05-21 01:15:34 I [<ipython-input-8-0bb789491088>:22] BiLSTMCRF(
  (embed): StaticEmbedding(
    (dropout_layer): Dropout(p=0, inplace=False)
    (embedding): Embedding(3471, 100, padding_idx=0)
  )
  (lstm): LSTM(
    (lstm): LSTM(100, 200, batch_first=True, bidirectional=True)
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=400, out_features=17, bi

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# 模型训练
batch_size = 32
n_epochs = 200
early_stopping = 10
trainer = Trainer(
    save_path=model_path,
    train_data=data_bundle.get_dataset('train'),
    model=model,
    loss=loss,
    optimizer=optimizer,
    batch_size=batch_size,
    n_epochs=n_epochs,
    dev_data=data_bundle.get_dataset('dev'),
    metrics=metric,
    metric_key='f',
    device=device,
    callbacks=[EarlyStopCallback(early_stopping)])
logger.warn('开始训练')
trainer.train()

2020-05-21 01:15:37 W [<ipython-input-10-d54362d8f065>:18] 开始训练


input fields after batch(if batch size is 2):
	target: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 26]) 
	seq_len: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 
	words: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 26]) 
target fields after batch(if batch size is 2):
	target: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 26]) 
	seq_len: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 

training epochs started 2020-05-21-01-15-37-473305


2020-05-21 01:15:41 W [callback.py:38] ======epoch : 1 , early stopping : 0/10======
2020-05-21 01:15:41 I [callback.py:40] metric_key : f, metric_value : 0.0
2020-05-21 01:15:41 I [callback.py:41] eval_result : 
{'SpanFPreRecMetric': {'f-gpe.nam': 0.0, 'pre-gpe.nam': 0.0, 'rec-gpe.nam': 0.0, 'f-org.nam': 0.0, 'pre-org.nam': 0.0, 'rec-org.nam': 0.0, 'f-per.nom': 0.0, 'pre-per.nom': 0.0, 'rec-per.nom': 0.0, 'f-gpe.nom': 0.0, 'pre-gpe.nom': 0.0, 'rec-gpe.nom': 0.0, 'f-loc.nam': 0.0, 'pre-loc.nam': 0.0, 'rec-loc.nam': 0.0, 'f-loc.nom': 0.0, 'pre-loc.nom': 0.0, 'rec-loc.nom': 0.0, 'f-org.nom': 0.0, 'pre-org.nom': 0.0, 'rec-org.nom': 0.0, 'f-per.nam': 0.0, 'pre-per.nam': 0.0, 'rec-per.nam': 0.0, 'f': 0.0, 'pre': 0.0, 'rec': 0.0}}


Evaluate data in 0.35 seconds!
Evaluation on dev at Epoch 1/200. Step:43/8600: 
SpanFPreRecMetric: f-gpe.nam=0.0, pre-gpe.nam=0.0, rec-gpe.nam=0.0, f-org.nam=0.0, pre-org.nam=0.0, rec-org.nam=0.0, f-per.nom=0.0, pre-per.nom=0.0, rec-per.nom=0.0, f-gpe.nom=0.0, pre-gpe.nom=0.0, rec-gpe.nom=0.0, f-loc.nam=0.0, pre-loc.nam=0.0, rec-loc.nam=0.0, f-loc.nom=0.0, pre-loc.nom=0.0, rec-loc.nom=0.0, f-org.nom=0.0, pre-org.nom=0.0, rec-org.nom=0.0, f-per.nam=0.0, pre-per.nam=0.0, rec-per.nam=0.0, f=0.0, pre=0.0, rec=0.0



2020-05-21 01:15:46 W [callback.py:38] ======epoch : 2 , early stopping : 0/10======
2020-05-21 01:15:46 I [callback.py:40] metric_key : f, metric_value : 0.0
2020-05-21 01:15:46 I [callback.py:41] eval_result : 
{'SpanFPreRecMetric': {'f-gpe.nam': 0.0, 'pre-gpe.nam': 0.0, 'rec-gpe.nam': 0.0, 'f-org.nam': 0.0, 'pre-org.nam': 0.0, 'rec-org.nam': 0.0, 'f-per.nom': 0.0, 'pre-per.nom': 0.0, 'rec-per.nom': 0.0, 'f-gpe.nom': 0.0, 'pre-gpe.nom': 0.0, 'rec-gpe.nom': 0.0, 'f-loc.nam': 0.0, 'pre-loc.nam': 0.0, 'rec-loc.nam': 0.0, 'f-loc.nom': 0.0, 'pre-loc.nom': 0.0, 'rec-loc.nom': 0.0, 'f-org.nom': 0.0, 'pre-org.nom': 0.0, 'rec-org.nom': 0.0, 'f-per.nam': 0.0, 'pre-per.nam': 0.0, 'rec-per.nam': 0.0, 'f': 0.0, 'pre': 0.0, 'rec': 0.0}}


...

2020-05-21 01:20:21 W [callback.py:38] ======epoch : 65 , early stopping : 9/10======
2020-05-21 01:20:21 I [callback.py:40] metric_key : f, metric_value : 0.505051
2020-05-21 01:20:21 I [callback.py:41] eval_result : 
{'SpanFPreRecMetric': {'f-gpe.nam': 0.352941, 'pre-gpe.nam': 0.529412, 'rec-gpe.nam': 0.264706, 'f-org.nam': 0.32, 'pre-org.nam': 0.631579, 'rec-org.nam': 0.214286, 'f-per.nom': 0.633609, 'pre-per.nom': 0.782313, 'rec-per.nom': 0.532407, 'f-gpe.nom': 0.0, 'pre-gpe.nom': 0.0, 'rec-gpe.nom': 0.0, 'f-loc.nam': 0.0, 'pre-loc.nam': 0.0, 'rec-loc.nam': 0.0, 'f-loc.nom': 0.105263, 'pre-loc.nom': 0.5, 'rec-loc.nom': 0.058824, 'f-org.nom': 0.0, 'pre-org.nom': 0.0, 'rec-org.nom': 0.0, 'f-per.nam': 0.481013, 'pre-per.nam': 0.542857, 'rec-per.nam': 0.431818, 'f': 0.505051, 'pre': 0.670498, 'rec': 0.405093}}


Evaluate data in 0.35 seconds!
Evaluation on dev at Epoch 65/200. Step:2795/8600: 
SpanFPreRecMetric: f-gpe.nam=0.352941, pre-gpe.nam=0.529412, rec-gpe.nam=0.264706, f-org.nam=0.32, pre-org.nam=0.631579, rec-org.nam=0.214286, f-per.nom=0.633609, pre-per.nom=0.782313, rec-per.nom=0.532407, f-gpe.nom=0.0, pre-gpe.nom=0.0, rec-gpe.nom=0.0, f-loc.nam=0.0, pre-loc.nam=0.0, rec-loc.nam=0.0, f-loc.nom=0.105263, pre-loc.nom=0.5, rec-loc.nom=0.058824, f-org.nom=0.0, pre-org.nom=0.0, rec-org.nom=0.0, f-per.nam=0.481013, pre-per.nam=0.542857, rec-per.nam=0.431818, f=0.505051, pre=0.670498, rec=0.405093



2020-05-21 01:20:25 W [callback.py:38] ======epoch : 66 , early stopping : 10/10======
2020-05-21 01:20:25 I [callback.py:40] metric_key : f, metric_value : 0.511955
2020-05-21 01:20:25 I [callback.py:41] eval_result : 
{'SpanFPreRecMetric': {'f-gpe.nam': 0.321429, 'pre-gpe.nam': 0.409091, 'rec-gpe.nam': 0.264706, 'f-org.nam': 0.293333, 'pre-org.nam': 0.578947, 'rec-org.nam': 0.196429, 'f-per.nom': 0.656, 'pre-per.nom': 0.773585, 'rec-per.nom': 0.569444, 'f-gpe.nom': 0.0, 'pre-gpe.nom': 0.0, 'rec-gpe.nom': 0.0, 'f-loc.nam': 0.0, 'pre-loc.nam': 0.0, 'rec-loc.nam': 0.0, 'f-loc.nom': 0.105263, 'pre-loc.nom': 0.5, 'rec-loc.nom': 0.058824, 'f-org.nom': 0.0, 'pre-org.nom': 0.0, 'rec-org.nom': 0.0, 'f-per.nam': 0.475, 'pre-per.nam': 0.527778, 'rec-per.nam': 0.431818, 'f': 0.511955, 'pre': 0.65233, 'rec': 0.421296}}
2020-05-21 01:20:25 I [callback.py:46] reach early stopping patience, stop training.
2020-05-21 01:20:25 I [callback.py:55] Early Stopping triggered in epoch 66!


Evaluate data in 0.34 seconds!
Reloaded the best model.

In Epoch:55/Step:2365, got best dev performance:
SpanFPreRecMetric: f-gpe.nam=0.4, pre-gpe.nam=0.625, rec-gpe.nam=0.294118, f-org.nam=0.30137, pre-org.nam=0.647059, rec-org.nam=0.196429, f-per.nom=0.662921, pre-per.nom=0.842857, rec-per.nom=0.546296, f-gpe.nom=0.0, pre-gpe.nom=0.0, rec-gpe.nom=0.0, f-loc.nam=0.0, pre-loc.nam=0.0, rec-loc.nam=0.0, f-loc.nom=0.0, pre-loc.nom=0.0, rec-loc.nom=0.0, f-org.nom=0.0, pre-org.nom=0.0, rec-org.nom=0.0, f-per.nam=0.496552, pre-per.nam=0.631579, rec-per.nam=0.409091, f=0.523169, pre=0.738397, rec=0.405093


{'best_epoch': 55,
 'best_eval': {'SpanFPreRecMetric': {'f': 0.523169,
   'f-gpe.nam': 0.4,
   'f-gpe.nom': 0.0,
   'f-loc.nam': 0.0,
   'f-loc.nom': 0.0,
   'f-org.nam': 0.30137,
   'f-org.nom': 0.0,
   'f-per.nam': 0.496552,
   'f-per.nom': 0.662921,
   'pre': 0.738397,
   'pre-gpe.nam': 0.625,
   'pre-gpe.nom': 0.0,
   'pre-loc.nam': 0.0,
   'pre-loc.nom': 0.0,
   'pre-org.nam': 0.647059,
   'pre-org.nom': 0.0,
   'pre-per.nam': 0.631579,
   'pre-per.nom': 0.842857,
   'rec': 0.405093,
   'rec-gpe.nam': 0.294118,
   'rec-gpe.nom': 0.0,
   'rec-loc.nam': 0.0,
   'rec-loc.nom': 0.0,
   'rec-org.nam': 0.196429,
   'rec-org.nom': 0.0,
   'rec-per.nam': 0.409091,
   'rec-per.nom': 0.546296}},
 'best_step': 2365,
 'seconds': 288.39}

In [11]:
# 评测
logger.warn('训练后评估')
tester = Tester(data=data_bundle.get_dataset('test'), model=model, metrics=metric, batch_size=64, device=device)
test_result = tester.test()
logger.info('评估结果：\n{}'.format(test_result))

2020-05-21 01:20:25 W [<ipython-input-11-2a8ad2ebd772>:2] 训练后评估


2020-05-21 01:20:26 I [<ipython-input-11-2a8ad2ebd772>:5] 评估结果：
{'SpanFPreRecMetric': {'f-gpe.nam': 0.416667, 'pre-gpe.nam': 0.6, 'rec-gpe.nam': 0.319149, 'f-org.nam': 0.115385, 'pre-org.nam': 0.230769, 'rec-org.nam': 0.076923, 'f-per.nom': 0.498339, 'pre-per.nom': 0.581395, 'rec-per.nom': 0.436047, 'f-gpe.nom': 0.0, 'pre-gpe.nom': 0.0, 'rec-gpe.nom': 0.0, 'f-loc.nam': 0.181818, 'pre-loc.nam': 0.666667, 'rec-loc.nam': 0.105263, 'f-org.nom': 0.0, 'pre-org.nom': 0.0, 'rec-org.nom': 0.0, 'f-loc.nom': 0.2, 'pre-loc.nom': 1.0, 'rec-loc.nom': 0.111111, 'f-per.nam': 0.467391, 'pre-per.nam': 0.605634, 'rec-per.nam': 0.380531, 'f': 0.421212, 'pre': 0.57438, 'rec': 0.332536}}


Evaluate data in 0.22 seconds!
[tester] 
SpanFPreRecMetric: f-gpe.nam=0.416667, pre-gpe.nam=0.6, rec-gpe.nam=0.319149, f-org.nam=0.115385, pre-org.nam=0.230769, rec-org.nam=0.076923, f-per.nom=0.498339, pre-per.nom=0.581395, rec-per.nom=0.436047, f-gpe.nom=0.0, pre-gpe.nom=0.0, rec-gpe.nom=0.0, f-loc.nam=0.181818, pre-loc.nam=0.666667, rec-loc.nam=0.105263, f-org.nom=0.0, pre-org.nom=0.0, rec-org.nom=0.0, f-loc.nom=0.2, pre-loc.nom=1.0, rec-loc.nom=0.111111, f-per.nam=0.467391, pre-per.nam=0.605634, rec-per.nam=0.380531, f=0.421212, pre=0.57438, rec=0.332536
